In [1]:
import scrapy

In [15]:
class Spider12(scrapy.Spider):
    #Le damos el nombre al scraper
    name = 'spider12'
    
    #Son los dominios en los que nos permite scrapear.. pueden ser varios elementos seguidos por una ','
    allowed_domains = ['pagina12.com.ar']
    
    #Nos permite configurar el archivo de salida
    custom_settings = {'FEEDS': 
                       {'pagina12.json':
                       {'format':'json',
                        'encoding':'utf-8'
                       }},
                       'DEPTH_LIMIT': 2}
    
    start_url = ['https://www.pagina12.com.ar/secciones/el-pais',
                 'https://www.pagina12.com.ar/secciones/economia',
                 'https://www.pagina12.com.ar/secciones/sociedad',
                 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                 'https://www.pagina12.com.ar/secciones/deportes',
                 'https://www.pagina12.com.ar/secciones/el-mundo',
                 'https://www.pagina12.com.ar/secciones/contratapa']
    
    def parse(self, response):
        # Articulo promocionado
        nota_promocionada = response.xpath('//div[@class="article-item__content"]/a/@href').get()
        if nota_promocionada is not None:
            yield response.follow(nota_promocionada, callback=self.parse_nota)
        
        # Listado de articulos
        notas = response.xpath('//div[@class="article-item__header"/a/@href]').get_all()
        for nota in notas:
            yield response.follow(nota, callback=self.parse_nota)
    
    
    def parse_nota(self, response):
    #Extract title
        title = response.xpath('//div[@class="article-titles"]/h1/text()').get()
    
     ##Extract body
            #This one is more difficult due to the html structure
        body = "".join(response.xpath('//div[@class="article-text"]/p/node()').get_all())
        body.replace('<p>', '').replace('</p>','').replace('<b>', '').replace('</b>','').replace('<br/>','').replace('&amp;','&')
   
        '''
    #Exract date
        date = response.xpath('//div[@class="article-info"]//div[@class="time"]/span/@datetime').get()
        
     #Extract intro
        intro = response.xpath('//div[@class="article-titles"]/h2/text()').get()
        if intro:
            
        else:
            
    # Extract Author
        author = response.xpath('//div[@class="article-main-media"]//div[@class="article-author"]/a/text()').get()
        '''
    # Link a la siguiente pagina de articulos
        next_page = response.xpath('//nav[@aria-label="Articles list pagination"]/a/@href')
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
        
        yield {'url':response.url,
               'titulo': title,
               'cuerpo': body}

In [9]:
from scrapy.crawler import CrawlerProcess

In [16]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

2020-12-07 21:39:45 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-07 21:39:45 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-12-07 21:39:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-12-07 21:39:45 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 2}
2020-12-07 21:39:45 [scrapy.extensions.telnet] INFO: Telnet Password: 7f4703c9853cb938
2020-12-07 21:39:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-12-07 21:39:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scr

ReactorNotRestartable: 